In [1]:
import random
from random import randint
import numpy as np 
import pandas as pd 
from faker import Faker
from faker.providers import BaseProvider

In [2]:
def random_with_N_digits(n):
    range_start = 10**(n-1)
    range_end = (10**n)-1
    return randint(range_start, range_end)

In [3]:
class CustomProvider:
    @classmethod
    def bdr_id(cls):
        faker = Faker()
        return faker.pystr_format("#########{{random_int}}") 

    @classmethod
    def iban_number(cls):
        faker = Faker()
        fake_iban = faker.country_code() + str(random_with_N_digits(2)) + faker.pystr(max_chars = 4).upper() + str(random_with_N_digits(6)) + str(random_with_N_digits(8))
        return fake_iban.upper() 
    
    @classmethod
    def id_class(cls):
        faker = Faker()
        return faker.pystr_format("#####{{random_int}}") 
    
    @classmethod
    def account_currency(cls):
        faker = Faker()
        return faker.pystr(max_chars = 3).upper()

    @classmethod
    def ten_letters(cls):
        faker = Faker()
        return faker.pystr(max_chars = 10).upper()
    
    @classmethod
    def two_letters(cls):
        faker = Faker()
        return faker.pystr(max_chars = 2).upper()
    
    @classmethod
    def one_letter(cls):
        faker = Faker()
        return faker.pystr(max_chars = 1).upper()
    
    @classmethod
    def num_compte(cls):
        faker = Faker()
        return faker.country_code() + str(random_with_N_digits(25))

In [8]:
faker = Faker()
faker.add_provider(CustomProvider())

In [14]:
faker.country_code()

'TO'

In [17]:
getattr(faker.unique, faker.country_code())

AttributeError: 'Generator' object has no attribute 'TZ'

In [24]:
test = []
for i in range(10000):
    test.append(getattr(faker.unique, 'iban_number')())

In [28]:
len(list(set(test)))

10000

In [4]:
def anonymizer(data, list_anonymised_dict):
    """
    Args:
        data (pandas.DataFrame)
            dataframe to anonymise
        list_anonymised_dict (list of dictionary)
            list of dictionary containing key as columns to anonymise & value as Faker category
    Returns:
        data (pandas.DataFrame)
            dataframe anonymised
        mapping (dictionary)
            mapping containing key as original value & value as anonymised value
    """
    mapping = {}
    for anonymised_dict in list_anonymised_dict:
        category = anonymised_dict['category']
        for col in anonymised_dict['col']:
            faker_unique_list = []
            unique_value = list((data[col]).unique())
            faker = Faker()
            faker.add_provider(CustomProvider())
            for i in range(len(unique_value)): 
                faker_unique_list.append(getattr(faker.unique, category)())
            mapping_anonymised = dict(zip(unique_value, faker_unique_list))
            mapping[col] = mapping_anonymised
            data[col] = data[col].map(mapping_anonymised)
    return data, mapping

In [5]:
def anonymizer_multi(data_dict, anonymised_list):
    """
    Args:
        data_dict (dictionary of pandas.DataFrame)
            dictionary of dataframe to anonymise
        anonymised_list (list of dictionary)
            list of dictionary containing key as columns to anonymise & value as Faker category
    Returns:
        data (pandas.DataFrame)
            dataframe anonymised
    """
    mapping_multi = {}
    for table_name in data_dict:
        mapping_multi[table_name] = {}
    for anonymised_dict in anonymised_list:
        faker = Faker()
        faker.add_provider(CustomProvider())        
        for col in anonymised_dict['col']:
            print('col', col)
            unique_value = []
            faker_unique_list = []
            for table_name in anonymised_dict['table_name_list']:
                print('table', table_name)
                unique_value += list((data_dict[table_name][col]).unique())

                for i in range(len(unique_value)): 
                    faker_unique_list.append(getattr(faker.unique, anonymised_dict['category'])())
            
            mapping_anonymised = dict(zip(unique_value, faker_unique_list))
            for table_name in anonymised_dict['table_name_list']:
                mapping_multi[table_name][col] = mapping_anonymised
                data_dict[table_name][col] = data_dict[table_name][col].map(mapping_anonymised)
    return data_dict, mapping_multi 

In [6]:
def anonymizer_iban_prepro(data, list_col):
    """
    Args:
        data (pandas.DataFrame)
        list_col (list of string)
            list of col to anonymize
    Returns:
        data anonymized (pandas.DataFrame)
    """
    for col in list_col:
        print('col', col)
        data[f'{col}_2_letters'] = data[col].str[:2]
        
    return data 

In [7]:
def anonymizer_iban_post(data, list_col):
    """
    Args:
        data (pandas.DataFrame)
        list_col (list of string)
            list of col to anonymize
    Returns:
        data anonymized (pandas.DataFrame)
    """
    for col in list_col:
        print('col', col)
        data[f'{col}_new'] = data[col]
        data = data.drop(columns = [col])
        data[col] = data[f'{col}_2_letters'] + data[f'{col}_new'].str[2:]
        print('Dropping col')
        data = data.drop(columns = [f'{col}_2_letters', f'{col}_new'])
    return data 